# Análisis de Siniestros Viales en Guadalajara y Zapopan (2019–2025)
## Notebook 01: Limpieza de datos y creación del GeoDataFrame

En este notebook se realiza la preparación inicial del conjunto de datos sobre siniestros viales ocurridos entre 2019 y 2025 en los municipios de Guadalajara y Zapopan.

El objetivo principal es dejar los datos listos para el análisis espacial. Para ello:

- Se carga el archivo original en formato CSV.
- Se renombran y validan las columnas que contienen coordenadas geográficas.
- Se eliminan registros sin latitud o longitud válida.
- Finalmente, se convierte el DataFrame en un `GeoDataFrame`, lo que permitirá visualizar los accidentes sobre mapas y realizar análisis geoespacial en los siguientes notebooks.

Este paso es fundamental para garantizar que el resto del análisis se realice sobre datos limpios y bien estructurados.




---

## 1. Carga y limpieza inicial de datos

En este primer paso, cargamos el conjunto de datos de accidentes viales ocurridos entre 2019 y 2025 en los municipios de Guadalajara y Zapopan.

- Usamos `pandas` para leer el archivo CSV con codificación `latin1` (común en archivos con acentos).
- Renombramos las columnas `x` e `y` a `lon` y `lat` para representar las coordenadas geográficas.
- Convertimos estas coordenadas a valores numéricos. Si alguna no es válida, se convierte a `NaN`.
- Finalmente, eliminamos los registros que no contienen coordenadas completas para asegurar que solo trabajaremos con datos geolocalizables.

Este paso nos prepara para crear un GeoDataFrame en la siguiente sección.


In [ ]:
import pandas as pd

df = pd.read_csv("../data/accidentes_zapopan_gdl_19_25_raw.csv", encoding="latin1")

# Renombrar columnas de coordenadas
df = df.rename(columns={"x": "lon", "y": "lat"})

# Convertir coordenadas a numéricas (los errores se vuelven NaN)
df["lon"] = pd.to_numeric(df["lon"], errors="coerce")
df["lat"] = pd.to_numeric(df["lat"], errors="coerce")

# Eliminar registros sin coordenadas válidas
df = df.dropna(subset=["lon", "lat"])

# Mostrar resumen del DataFrame
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 14493 entries, 2 to 16181
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         14493 non-null  int64  
 1   consecutivo        14493 non-null  int64  
 2   Id                 14493 non-null  object 
 3   fecha              14493 non-null  object 
 4   anio               14493 non-null  object 
 5   mes                14493 non-null  object 
 6   dia                14493 non-null  object 
 7   dia_sem            14493 non-null  object 
 8   rango_hora         14493 non-null  object 
 9   mun                14493 non-null  object 
 10  calle_1            14493 non-null  object 
 11  calle_2            14493 non-null  object 
 12  lon                14493 non-null  float64
 13  lat                14493 non-null  float64
 14  tipo_siniestro     14493 non-null  object 
 15  condicion_usuario  14493 non-null  object 
 16  tipo_usuario       14493 no

,Unnamed: 0,consecutivo,Id,fecha,anio,mes,dia,dia_sem,rango_hora,mun,...,calle_2,lon,lat,tipo_siniestro,condicion_usuario,tipo_usuario,ibaen_atro,sexo,rango_edad,consecuencia
2,3,26558,2019-01-01-100002-EHGR-22:00 A 23:59,2019-01-01,2019,Enero,1,martes,23:00 a 23:59,Guadalajara,...,Javier Mina,-103.3161,20.66837,Colision Entre Dos o Mas Vehiculos,Conductor,Transporte Empresarial O De Personal,Camioneta De Pasajeros,No especificado,No disponible,Ileso
3,4,26559,2019-01-01-100002-EHGR-22:00 A 23:59,2019-01-01,2019,Enero,1,martes,23:00 a 23:59,Guadalajara,...,Javier Mina,-103.3161,20.66837,Colision Entre Dos o Mas Vehiculos,Conductor,Transporte de Carga,Camioneta De Carga,No especificado,No disponible,Lesionado
4,5,26560,2019-01-01-53946-SISCAV-04:00 A 05:59,2019-01-01,2019,Enero,1,martes,05:00 a 05:59,Guadalajara,...,Calle 5 Zi,-103.3648,20.63889,Colision Entre Dos o Mas Vehiculos,Conductor,Vehiculo Particular,Automovil,Hombre,28-37,Ileso
5,6,26561,2019-01-01-53946-SISCAV-04:00 A 05:59,2019-01-01,2019,Enero,1,martes,05:00 a 05:59,Guadalajara,...,Calle 5 Zi,-103.3648,20.63889,Colision Entre Dos o Mas Vehiculos,Conductor,Vehiculo Particular,Automovil,Hombre,18-27,Lesionado
6,7,26562,2019-01-01-53946-SISCAV-04:00 A 05:59,2019-01-01,2019,Enero,1,martes,05:00 a 05:59,Guadalajara,...,Calle 5 Zi,-103.3648,20.63889,Colision Entre Dos o Mas Vehiculos,Conductor,Transporte de Carga,Camion De Carga,Hombre,48-59,Ileso


---

## 2. Creación del GeoDataFrame

Ahora que ya tenemos las coordenadas limpias y en formato numérico, el siguiente paso es convertir nuestro DataFrame en un GeoDataFrame. Esto nos permitirá trabajar con geometrías y hacer análisis espacial directamente en Python.

Para ello usamos la función `points_from_xy()` de `geopandas`, que toma las columnas de longitud (`lon`) y latitud (`lat`) y las convierte en puntos geográficos.

También especificamos el sistema de referencia de coordenadas `EPSG:4326`, que es el estándar para latitud y longitud (WGS 84).

A partir de este punto, ya podemos empezar a visualizar los siniestros sobre un mapa y realizar análisis geoespacial.


In [2]:
import geopandas as gpd

# Crear GeoDataFrame a partir de latitud y longitud
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["lon"], df["lat"]),
    crs="EPSG:4326"  # Sistema de coordenadas WGS 84 (lat/lon)
)

gdf.head()


,Unnamed: 0,consecutivo,Id,fecha,anio,mes,dia,dia_sem,rango_hora,mun,...,lon,lat,tipo_siniestro,condicion_usuario,tipo_usuario,ibaen_atro,sexo,rango_edad,consecuencia,geometry
2,3,26558,2019-01-01-100002-EHGR-22:00 A 23:59,2019-01-01,2019,Enero,1,martes,23:00 a 23:59,Guadalajara,...,-103.3161,20.66837,Colision Entre Dos o Mas Vehiculos,Conductor,Transporte Empresarial O De Personal,Camioneta De Pasajeros,No especificado,No disponible,Ileso,POINT (-103.3161 20.66837)
3,4,26559,2019-01-01-100002-EHGR-22:00 A 23:59,2019-01-01,2019,Enero,1,martes,23:00 a 23:59,Guadalajara,...,-103.3161,20.66837,Colision Entre Dos o Mas Vehiculos,Conductor,Transporte de Carga,Camioneta De Carga,No especificado,No disponible,Lesionado,POINT (-103.3161 20.66837)
4,5,26560,2019-01-01-53946-SISCAV-04:00 A 05:59,2019-01-01,2019,Enero,1,martes,05:00 a 05:59,Guadalajara,...,-103.3648,20.63889,Colision Entre Dos o Mas Vehiculos,Conductor,Vehiculo Particular,Automovil,Hombre,28-37,Ileso,POINT (-103.3648 20.63889)
5,6,26561,2019-01-01-53946-SISCAV-04:00 A 05:59,2019-01-01,2019,Enero,1,martes,05:00 a 05:59,Guadalajara,...,-103.3648,20.63889,Colision Entre Dos o Mas Vehiculos,Conductor,Vehiculo Particular,Automovil,Hombre,18-27,Lesionado,POINT (-103.3648 20.63889)
6,7,26562,2019-01-01-53946-SISCAV-04:00 A 05:59,2019-01-01,2019,Enero,1,martes,05:00 a 05:59,Guadalajara,...,-103.3648,20.63889,Colision Entre Dos o Mas Vehiculos,Conductor,Transporte de Carga,Camion De Carga,Hombre,48-59,Ileso,POINT (-103.3648 20.63889)


## Guardado de datos limpios

Una vez finalizada la limpieza y transformación de los datos, guardamos los resultados para reutilizarlos fácilmente en los siguientes notebooks del proyecto.

- Se guarda un archivo `.csv` sin geometría, útil para análisis exploratorio y estadísticas.
- También se guarda un `.geojson` con geometría incluida, ideal para análisis espacial y visualizaciones en mapas.


In [4]:
# Guardar CSV limpio sin geometría
gdf.drop(columns="geometry").to_csv("../data/accidentes_cleaned.csv", index=False)

# GeoJSON para mantener geometría
gdf.to_file("../data/accidentes_shapes.geojson", driver="GeoJSON")
